how much data should we collect. clean it so that the non images data points are gone

train lora model
train prompt tuning model
evaluation metrics to compare lora, prompt tuning, and no finetuning

In [ ]:
import os
os.getcwd()
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Final_Project'

/content/drive/.shortcut-targets-by-id/17G1FGpU_bcNdWQVtbe6Brhc0xIk4ybzC/Final_Project


In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets
!pip install httpx

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 229.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('Contrastive.csv')
df.head(5)

,emotion,utterance,art_style,painting,anchor_art_style,anchor_painting,repetition
0,sadness,This painting shows a gloomy day. The darkness...,Realism,johan-hendrik-weissenbruch_windy-day,Realism,johan-hendrik-weissenbruch_ship-canal,1
1,sadness,the weather and atmosphere makes me feel sad,Romanticism,ivan-aivazovsky_landscape-with-a-schooner-1880,Realism,johan-hendrik-weissenbruch_ship-canal,45
2,sadness,"Dark and moody, cloudy sky that is depressing",Impressionism,paul-gauguin_clearing-1873,Realism,johan-hendrik-weissenbruch_ship-canal,18
3,sadness,the picture is dark with a somber tone,Impressionism,john-henry-twachtman_winter-landscape,Realism,johan-hendrik-weissenbruch_ships-in-harbour,7
4,sadness,the dark colors make the painting look gloomy,Impressionism,eugene-boudin_trouville-the-port-1864,Realism,johan-hendrik-weissenbruch_ships-in-harbour,5


In [ ]:
# data exploration
print('total unique anchor paintings: ',len(set(df['anchor_painting'])))
print('total unique paintings: ',len(set(df['painting'])))
print('total unique art styles: ',len(set(df['art_style'])))
print('unique emotions: ',set(df['emotion']))

total unique anchor paintings:  49824
total unique paintings:  50905
total unique art styles:  27
unique emotions:  {'awe', 'disgust', 'amusement', 'anger', 'fear', 'excitement', 'contentment', 'sadness'}


In [ ]:
# download the image files themselves (the original ZIP FILE IS 25G so we'll scrape a small subset lol)
import httpx
from bs4 import BeautifulSoup

base_url = "https://uploads8.wikiart.org/images/"
stopper = 25
valid_images_count = 0

for item in df.iterrows():
    item_name_spl = item[1]['painting'].split('_')
    painter = item_name_spl[0]
    painting = item_name_spl[1]
    scrape_url = base_url + painter + '/' + painting + '.jpg!Large.jpg'

    try:
        response = httpx.get(scrape_url)
        if response.status_code == 200 and response.headers['Content-Type'].startswith('image'):
            with open('imgs/' + item[1]['painting'] + '.jpeg', 'wb') as outf:
                outf.write(response.content)
            valid_images_count += 1
            print(f"Downloaded image {valid_images_count}: {item[1]['painting']}")
        else:
            print(f"Skipped invalid image URL: {scrape_url}")
    except httpx.RequestError as e:
        print(f"Error downloading image: {scrape_url}")
        print(f"Error message: {str(e)}")

    if valid_images_count == stopper:
        break

KeyboardInterrupt: 

process data

In [ ]:
print(df.iloc[0]['painting'])

johan-hendrik-weissenbruch_windy-day


In [ ]:
# define a dataset class
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class ArtemisDataset(Dataset):
    def __init__(self, processor, labels_fname="Contrastive.csv", img_dirname="imgs/"):
        self.img_labels = pd.read_csv(labels_fname).iloc[:24]
        self.img_dirname = img_dirname
        self.processor = processor

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        painting_name = self.img_labels.iloc[idx]['painting'] + '.jpeg'
        img_path = os.path.join(self.img_dirname, painting_name)
        image = read_image(img_path)
        label = self.img_labels.iloc[idx]['utterance']
        emotion = self.img_labels.iloc[idx]['emotion']

        label= f"This is a caption that invokes {emotion}: {label}"

        encoding = self.processor(images=image, padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = label
        return encoding

def collate_fn(batch):
  # pad the input_ids and attention_mask
  processed_batch = {}
  for key in batch[0].keys():
      if key != "text":
          processed_batch[key] = torch.stack([example[key] for example in batch])
      else:
          text_inputs = processor.tokenizer(
              [example["text"] for example in batch], padding=True, return_tensors="pt"
          )
          processed_batch["input_ids"] = text_inputs["input_ids"]
          processed_batch["attention_mask"] = text_inputs["attention_mask"]
  return processed_batch



load the model and img processor

In [ ]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model-00005-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/782M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, lora_config)

In [ ]:
train_dataset = ArtemisDataset(processor=processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=5, collate_fn=collate_fn)

In [ ]:
import torch

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()
for epoch in range(5):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device, torch.float16)
    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)
    loss = outputs.loss
    print("Loss:", loss.item())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Epoch: 0
Loss: 1.951171875
Loss: 1.7041015625
Loss: 1.8759765625
Loss: 1.59765625
Loss: 1.595703125
Epoch: 1
Loss: 1.736328125
Loss: 1.630859375
Loss: 1.341796875
Loss: 1.2578125
Loss: 1.45703125
Epoch: 2
Loss: 1.2255859375
Loss: 1.3984375
Loss: 1.51171875
Loss: 1.44921875
Loss: 1.330078125
Epoch: 3
Loss: 1.1318359375
Loss: 1.5068359375
Loss: 1.0
Loss: 1.1982421875
Loss: 1.154296875
Epoch: 4
Loss: 1.6259765625
Loss: 1.1826171875
Loss: 1.03125
Loss: 0.9150390625
Loss: 1.162109375


In [ ]:
prompt = "This is a caption that invokes sadness: "
img_path= 'imgs/alfred-sisley_bank-in-the-autumn-1879.jpeg'
image = read_image(img_path)

inputs = processor(images=image, text=prompt, return_tensors="pt").to(device="cuda", dtype=torch.float16)

generated_ids = model.generate(**inputs, max_length=80)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

~~~The picture is dark and gloomy. The trees are bare and the sky is gloomy. The trees are bare and the sky is gloomy.~~~~~~~~~~~~~~~~
